In [8]:
import requests
import json


In [9]:
def fetch_and_transform_season(season, start_id):
    url = f"https://raw.githubusercontent.com/openfootball/football.json/master/{season}/en.1.json"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors du téléchargement de la saison {season}")
        return [], start_id

    data = response.json()
    matches = data.get("matches", [])
    transformed = []
    template = "the score of the match between [team1] and [team2] the [date] was {}"

    for match in matches:
        team1 = match["team1"]
        team2 = match["team2"]
        date = match["date"]
        score = match.get("score", {}).get("ft")
        if not score:
            continue  # Skip matches without full-time score
        score_str = f"{score[0]}-{score[1]}"
        prompt = f"the score of the match between {team1} and {team2} the {date} was"

        entry = {
            "known_id": start_id,
            "subject": team1,
            "attribute": f" {score_str}",  # Leading space for tokenization
            "template": template,
            "prediction": None,
            "prompt": prompt,
            "relation_id": "match_score"
        }
        transformed.append(entry)
        start_id += 1

    return transformed, start_id

# Liste des saisons à traiter
seasons = ["2010-11", "2011-12", "2012-13", "2013-14", "2014-15"]

all_data = []
current_id = 0

for season in seasons:
    season_data, current_id = fetch_and_transform_season(season, current_id)
    all_data.extend(season_data)

# Sauvegarde des données fusionnées dans un fichier JSON
with open("premier_league_2010_2015.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)

print(f"Total de matchs traités : {len(all_data)}")


Total de matchs traités : 1900


In [7]:
all_data

[{'known_id': 0,
  'subject': 'Tottenham Hotspur',
  'attribute': ' 0-0',
  'template': 'the score of the match between [team1] and [team2] the [date] was {}',
  'prediction': None,
  'prompt': 'the score of the match between Tottenham Hotspur and Manchester City the 2010-08-14 was',
  'relation_id': 'match_score'},
 {'known_id': 1,
  'subject': 'Wigan Athletic',
  'attribute': ' 0-4',
  'template': 'the score of the match between [team1] and [team2] the [date] was {}',
  'prediction': None,
  'prompt': 'the score of the match between Wigan Athletic and Blackpool FC the 2010-08-14 was',
  'relation_id': 'match_score'},
 {'known_id': 2,
  'subject': 'Bolton Wanderers',
  'attribute': ' 0-0',
  'template': 'the score of the match between [team1] and [team2] the [date] was {}',
  'prediction': None,
  'prompt': 'the score of the match between Bolton Wanderers and Fulham FC the 2010-08-14 was',
  'relation_id': 'match_score'},
 {'known_id': 3,
  'subject': 'Wolverhampton Wanderers',
  'att